# Project - Airline AI Assistant

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline

In [25]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import anthropic

In [65]:
system_message = """
You are a native Japanese tourist guide. Provide foreign tourists with high-quality guidance for the best Japan experience. Avoid tourist traps by prioritizing recommendations based on native opinions, rather than those popular only with foreign visitors. Do not be influenced by biased advertisements; instead, seek trustworthy, well-reviewed options. Respond using Markdown formatting.

Research reputable sources to find the best prices for tickets and provide relevant links. If suitable links or prices cannot be found, state this clearly.

## Output Format
Present your guidance in Markdown with the following structure:

### 1. Overview
- Brief introduction to the recommended activities/locations.

### 2. Recommendations
- Use section headings (e.g., **Sightseeing**, **Food & Drink**, **Accommodation**).
- For each recommendation, include:
  - Name and brief description
  - Why it is valued by locals
  - [If applicable] Link to ticket/official website (in Markdown link format)
  - Best available price or note if not available

### 3. Additional Tips
- Tips for avoiding tourist traps
- Insider advice from local perspectives

If you cannot find reliable ticket links or prices, include a note such as:
> Suitable ticket link or price information could not be found at this time.
"""

In [26]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()
claude = anthropic.Anthropic()

# As an alternative, if you'd like to use Ollama instead of OpenAI
# Check that Ollama is running for you locally (see week1/day2 exercise) then uncomment these next 2 lines
# MODEL = "llama3.2"
# openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')


OpenAI API Key exists and begins sk-proj-


In [66]:
def stream_gpt(system, prompt, history):
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": prompt}]
    print(messages)
    stream = openai.responses.create(
        model='gpt-4o-mini',
        input=messages,
        tools=[{"type": "web_search_preview"}],
        stream=True,
        temperature=1.4
    )
    result = ""
    for chunk in stream:
        if chunk.type == "response.output_text.delta":
            result += chunk.delta or ""
            yield result
def stream_claude(system, prompt, history):
    messages = history + [{"role": "user", "content": prompt}]
    response = ""
    with claude.messages.stream(
        model="claude-sonnet-4-20250514",
        max_tokens=2000,
        temperature=0.7,
        system=system,
        messages=messages,
        tools=[{
        "type": "web_search_20250305",
        "name": "web_search",
        "max_uses": 10
        }]
    ) as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response
    print(response)

from functools import partial
def stream_model_factory(model, system):
    if model=="GPT":
        return partial(stream_gpt, system)
    elif model=="Claude":
        return partial(stream_claude, system)
    else:
        raise ValueError("Unknown model")
def chat(prompt, history, model):
    func = stream_model_factory(model, system_message)
    history = [{"content": hist["content"], "role": hist["role"]} for hist in history]
    stream = func(prompt, history)
    for chunk in stream:
        yield chunk

In [67]:
with gr.Blocks() as ui:
    model = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")
    gr.ChatInterface(fn=chat, type="messages", additional_inputs=model)
ui.launch()

* Running on local URL:  http://127.0.0.1:7891
* To create a public link, set `share=True` in `launch()`.


[{'role': 'system', 'content': '\nYou are a native Japanese tourist guide. Provide foreign tourists with high-quality guidance for the best Japan experience. Avoid tourist traps by prioritizing recommendations based on native opinions, rather than those popular only with foreign visitors. Do not be influenced by biased advertisements; instead, seek trustworthy, well-reviewed options. Respond using Markdown formatting.\n\nResearch reputable sources to find the best prices for tickets and provide relevant links. If suitable links or prices cannot be found, state this clearly.\n\n## Output Format\nPresent your guidance in Markdown with the following structure:\n\n### 1. Overview\n- Brief introduction to the recommended activities/locations.\n\n### 2. Recommendations\n- Use section headings (e.g., **Sightseeing**, **Food & Drink**, **Accommodation**).\n- For each recommendation, include:\n  - Name and brief description\n  - Why it is valued by locals\n  - [If applicable] Link to ticket/off

## Tools

Tools are an incredibly powerful feature provided by the frontier LLMs.

With tools, you can write a function, and have the LLM call that function as part of its response.

Sounds almost spooky.. we're giving it the power to run code on our machine?

Well, kinda.

In [5]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [6]:
get_ticket_price("Berlin")

Tool get_ticket_price called for Berlin


'$499'

In [7]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [ ]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function}]

## Getting OpenAI to use our Tool

There's some fiddly stuff to allow OpenAI "to call our tool"

What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool.

Here's how the new chat function looks:

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [ ]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, city

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()